In [2]:
#import json
import ijson

In [7]:
#to test json parsing

with open("./reddit-tomt-submissions.jsonl", "r") as f:
    for record in ijson.items(f, "item"):
        id = record["id"]
        title = record["title"]
        if id == "2flpx1":
            print(title)
            #should be "[TOMT] Name of arcade game I love"

IncompleteJSONError: lexical error: invalid char in json text.
          se, "is_self": true, "from": NaN, "author": "TwoBonesJones",
                     (right here) ------^


In [ ]:
#actual parsing

with open("./reddit-tomt-submissions.jsonl", "r") as f:
    for record in ijson.items(f, "item"): #goes through json items and 
        solved = record["solved_utc"] != "" #tests if topic is answered
        answer_time = record["solved_utc"] - record["created_utc"]
        upvotes = record["ups"]
        downvotes = record["downs"]
        comments = record["num_comments"]